In [2]:
import os
import torch
import gpytorch
import numpy as np
from torch import nn
from torch import matmul as m
from gpytorch.kernels import RBFKernel, ScaleKernel
from torch.distributions import Normal, Poisson, MultivariateNormal

from deep_fields import project_path
from matplotlib import pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from deep_fields.models.utils.basic_setups import create_dir_and_writer
from deep_fields.models.gaussian_processes.gaussian_processes import multivariate_normal, white_noise_kernel

\begin{equation}
d\mathbf{x}_t = \mu(\mathbf{x}_t)dt + \sqrt{\Sigma(\mathbf{x}_t)}dW_t
\end{equation}

\begin{eqnarray}
\mu(\mathbf{x}) = K_{\mathbf{x},Z_f}K^{-1}_{Z_f Z_f}\text{vec}(U_f) \\
\Sigma(\mathbf{x}) = K_{\mathbf{x}\mathbf{x}} - K_{\mathbf{x},Z_f}K^{-1}_{Z_f Z_f}K_{Z_f,\mathbf{x}} \\
p(U_f) = \prod^{D}_{d=1}\mathcal{N}(0,K_{f_d,f_d}) \\
K_{Z_f Z_f} \in \mathbb{R}^{MD\times MD}
\end{eqnarray}

In [3]:
class GPModel(nn.Module):
    
    def __init__(self,kernel_sigma,kernel_lenght_scales,locations_dimension=1):
        nn.Module.__init__(self)        
        kernel = ScaleKernel(RBFKernel(ard_num_dims=locations_dimension, requires_grad=True),
                     requires_grad=True) + white_noise_kernel()
        kernel_hypers = {"raw_outputscale": torch.tensor(kernel_sigma),
                         "base_kernel.raw_lengthscale": torch.tensor(kernel_lenght_scales)}
        kernel.kernels[0].initialize(**kernel_hypers)
        
        self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = kernel

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x).evaluate()
        return mean_x,covar_x
    
    def calculate_posterior(test_input,train_output,train_input,output_beta=None):
        """
        :param test_input: torch tensor
        :param train_input: torch tensor
        :param kernel:
        :return: predictive_mean [number_of_points], predictive_variance [number_of_points]
        """
        """

        :param test_input: torch tensor
        :param train_input: torch tensor
        :param kernel:
        :return: predictive_mean [number_of_points], predictive_variance [number_of_points]
        """

        K_train_train = self.covar_module.forward(train_input, train_input)
        K_test_train = self.covar_module.forward(test_input, train_input)
        K_test_test = self.covar_module.forward(test_input, test_input, diag=True)

        if output_beta is None:
            K_train_train = K_train_train.evaluate()
        else:
            K_train_train = K_train_train.evaluate() + torch.tensor(1./output_beta * np.eye(len(train_input)))

        prior_mean = self.mean_module(train_input)
        prior_normal = multivariate_normal(prior_mean , K_train_train)

        K_train_train_inverse = prior_normal.varinv()
        kappa = K_test_train.evaluate().matmul(K_train_train_inverse).double()

        predictive_mean = kappa.matmul(train_output.double().T).double()

        predictive_variance = kappa.matmul(K_test_train.evaluate().T)
        predictive_variance = predictive_variance.diag()
        predictive_variance = K_test_test - predictive_variance

        return predictive_mean.T[0], predictive_variance

# Inducing Prior

In [4]:
locations_dimension = 2
kernel_parameters = {"kernel_sigma": .1,
                     "kernel_lenght_scales": [0.1, 0.1]}
number_of_time_steps = 50
        
kwargs = {"locations_dimension": locations_dimension,
          "total_assets_in_history":100,
          "number_of_inducing_points":10,
          "jump_size_scale_prior": 1.,
          "birth_intensity":4.,
          "returns_mean_a": 1.,
          "returns_mean_b": 1.,
          "prior_locations_mean": 0.,
          "prior_locations_std": 1.,
          "prior_sigma_mean": 0.,
          "prior_sigma_std": 1.,
          "prior_length_mean": 0.,
          "prior_length_std": 1.,
          "kernel_parameters": kernel_parameters,
          "number_of_realizations": number_of_time_steps,
          "model_path": os.path.join(project_path, 'results')}
                  
locations_dimension = kwargs.get("locations_dimension")
prior_locations_mean = kwargs.get("prior_locations_mean")
prior_locations_std = kwargs.get("prior_locations_std")

kernel_sigma = kernel_parameters.get("kernel_sigma")
kernel_lenght_scales = kernel_parameters.get("kernel_lenght_scales")

total_assets_in_history = kwargs.get("total_assets_in_history")
number_of_inducing_points = kwargs.get("number_of_inducing_points")

# Expected Returns
returns_mean_a = kwargs.get("returns_mean_a")
returns_mean_b = kwargs.get("returns_mean_b")

# Locations Prior
locations_prior = Normal(torch.full((locations_dimension,), prior_locations_mean),
                         torch.full((locations_dimension,), prior_locations_std))

# Mean Function
mean_module = gpytorch.means.ZeroMean()

# Kernels Prior
kernel = ScaleKernel(RBFKernel(ard_num_dims=locations_dimension, requires_grad=True),
             requires_grad=True) + white_noise_kernel()
kernel_hypers = {"raw_outputscale": torch.tensor(kernel_sigma),
                 "base_kernel.raw_lengthscale": torch.tensor(kernel_lenght_scales)}
kernel.kernels[0].initialize(**kernel_hypers)

ScaleKernel(
  (base_kernel): RBFKernel(
    (raw_lengthscale_constraint): Positive()
  )
  (raw_outputscale_constraint): Positive()
)

In [5]:
locations_total = locations_prior.sample(sample_shape=(total_assets_in_history,))
locations_inducing_points = locations_prior.sample(sample_shape=(number_of_inducing_points,))

In [28]:
K_inducing_inducing = kernel.forward(locations_inducing_points, locations_inducing_points)
K_total_inducing = kernel.forward(locations_total, locations_inducing_points)
K_total_total = kernel.forward(locations_total, locations_total)

K_total_total = K_total_total.evaluate()
K_total_inducing = K_total_inducing.evaluate()
K_inducing_inducing = K_inducing_inducing.evaluate()

In [32]:
# Inducing Function
inducing_mean = mean_module(train_input)
inducing_distribution = multivariate_normal(inducing_mean , K_train_train)

In [36]:
K_inducing_inducing_inverse = inducing_distribution.varinv()
kappa = K_total_inducing.matmul(K_inducing_inducing_inverse)
total_variance = kappa.matmul(K_total_inducing.T)
total_variance =  K_total_total - total_variance

In [34]:
# sample returns
returns_mean_prior = torch.zeros(total_assets_in_history) * returns_mean_a
returns_covariance_prior = returns_mean_b * predictive_variance
expected_returns_distribution = multivariate_normal(returns_mean_prior, returns_covariance_prior)
expected_returns = expected_returns_distribution.sample()

In [39]:
#simulate process
U_f = inducing_distribution.sample()
total_mean = m(kappa,U_f)
process_distribution = multivariate_normal(expected_returns+total_mean, returns_covariance_prior)

In [79]:
process_sample = process_distribution.sample(sample_shape=torch.Size([number_of_time_steps]))

In [69]:
# posterior training
mean_u = torch.Tensor(np.random.random(number_of_inducing_points))
mean_u = torch.nn.Parameter(mean_u)

Sigma_u = torch.Tensor(np.random.random(number_of_inducing_points))
Sigma_u = torch.nn.Parameter(Sigma_u)

epsilon_distribution = Normal(torch.zeros_like(mean_u), torch.ones_like(mean_u))
posterior_distribution = Normal(mean_u,Sigma_u**2)

In [64]:
#reparametrization trick
sampled_u = mean_u + Sigma_u*epsilon_distribution.sample()

In [72]:
sampled_mean = m(kappa,sampled_u)
likelihood = multivariate_normal(expected_returns+sampled_mean, returns_covariance_prior)

In [84]:
LIKELIHOOD = likelihood.log_prob(process_sample)

In [112]:
log_det = torch.log(posterior_distribution.scale.prod()/torch.det(inducing_distribution.covariance_matrix))
trace_ = torch.trace(m(K_inducing_inducing_inverse,torch.diag(Sigma_u)))
means_ = m(mean_u,m(K_inducing_inducing_inverse,mean_u))
kl = log_det + trace_ + means_

In [116]:
ELBO = LIKELIHOOD.sum() - kl